In [1]:
import re
import datetime
import pandas as pd
import requests 
import os

In [3]:
#pip install mysql-connector-python
# pip install python-dotenv
import mysql.connector
conexao = mysql.connector.connect(host='localhost',user='root',password='',database='producao')
cursor = conexao.cursor()

In [4]:
from dotenv import load_dotenv
import os
from openai import OpenAI
from langchain_openai import ChatOpenAI

# pip install openai
# pip install langchain-openai
# saldo disponível
# https://platform.openai.com/settings/organization/billing/overview
# chaves API-KEY
# https://platform.openai.com/api-keys

In [5]:
load_dotenv(dotenv_path='.env', override=True)

True

In [6]:
openai_api_key = os.getenv("OPENAI_API_KEY")

In [7]:
question = "Qual a capital do Brasil?"
llm = ChatOpenAI(model="gpt-4o-mini", openai_api_key = openai_api_key, temperature=0.7,max_tokens=256)
messages=[{"role":"user", "content": question}]
response = llm.invoke(messages)
print(response.content)

A capital do Brasil é Brasília.


In [8]:
# pip install tabulate
numero = "PI0808715"
# atualiza no localhost as tres tabelas: carga, anterioridades e anterioridades_desc
comando = f"SELECT * FROM arquivados WHERE numero='{numero}'"
cursor.execute(comando)
resultado = cursor.fetchall()
df = pd.DataFrame(resultado)
print(df.to_markdown(index=False)) 

|       0 | 1             | 2         | 3          | 4     |   5 |   6 |
|--------:|:--------------|:----------|:-----------|:------|----:|----:|
| 1011532 | 1.1           | PI0808715 | 2011-08-09 | dialp |   0 |   0 |
| 1475660 | 1.3           | PI0808715 | 2014-08-12 | dialp |   0 |   0 |
| 1501853 | 6.6           | PI0808715 | 2014-09-16 | dialp |   0 |   0 |
| 2788968 | 7.1           | PI0808715 | 2017-04-18 | dialp |   0 |   1 |
| 2790019 | 15.11         | PI0808715 | 2017-04-18 | dialp |   0 |   0 |
| 2956330 | 9.2           | PI0808715 | 2017-09-19 | dialp |   0 |   2 |
| 3015281 | 12.2          | PI0808715 | 2017-12-19 | dialp |   0 |   0 |
| 4414189 | PR - Recursos | PI0808715 | 2022-01-04 | dialp |   0 |   0 |
| 4551344 | PR - Recursos | PI0808715 | 2022-08-16 |       |   0 |   0 |


In [9]:
df

,0,1,2,3,4,5,6
0,1011532,1.1,PI0808715,2011-08-09,dialp,0,0
1,1475660,1.3,PI0808715,2014-08-12,dialp,0,0
2,1501853,6.6,PI0808715,2014-09-16,dialp,0,0
3,2788968,7.1,PI0808715,2017-04-18,dialp,0,1
4,2790019,15.11,PI0808715,2017-04-18,dialp,0,0
5,2956330,9.2,PI0808715,2017-09-19,dialp,0,2
6,3015281,12.2,PI0808715,2017-12-19,dialp,0,0
7,4414189,PR - Recursos,PI0808715,2022-01-04,dialp,0,0
8,4551344,PR - Recursos,PI0808715,2022-08-16,,0,0


In [10]:
for item in resultado:
    numero = item[0]
    despacho = item[1]
    print(f"Número: {numero} | Despacho: {despacho}")

Número: 1011532 | Despacho: 1.1
Número: 1475660 | Despacho: 1.3
Número: 1501853 | Despacho: 6.6
Número: 2788968 | Despacho: 7.1
Número: 2790019 | Despacho: 15.11
Número: 2956330 | Despacho: 9.2
Número: 3015281 | Despacho: 12.2
Número: 4414189 | Despacho: PR - Recursos
Número: 4551344 | Despacho: PR - Recursos


In [14]:
comando = f"SELECT * FROM titulo WHERE titulo_cepit<>'' and resumo_cepit like '%Abstract%'"
cursor.execute(comando)
resultado = cursor.fetchall()
df = pd.DataFrame(resultado)
total = 0

for item in resultado:
    total = total + 1
    if (total>10):
        break
    numero = item[1]
    titulo_cepit = item[5]
    resumo_cepit = item[6]
    #print(f"Título: {titulo_cepit}")
    #print(f"Resumo: {resumo_cepit}\n")

    question = f"""Elimine os trechos em inglês. Converta para minúsculas mantendo a primeira letra de cada frase em maiúsculo.
        Elimine os termos 'Resumo' e 'Patente de invenção' no início. Siglas devem ser mantidas na forma original, por exemplo 
        PBCH. Mostre como resposta apenas o texto em português. Se todo o texto estiver em inglês não mostre nada: O texto a analisar é o seguinte: {resumo_cepit}"""
    messages=[{"role":"user", "content": question}]
    response = llm.invoke(messages)
    #print("\033[34m" + response.content + "\033[0m")
    resumo = response.content
    resumo = resumo.replace('"', '')
    resumo = resumo.replace("'", '')
    cmd = f"update titulo set resumo_cepit='{resumo}' where numero='{numero}'"
    print(cmd + ';')
    #print('#################')

update titulo set resumo_cepit='A presente invenção refere-se a um artigo que está revestido com um revestimento que tem uma cor escura. Em uma modalidade preferida, o revestimento compreende uma camada de revestimento de base de níquel ou polímero, e uma primeira camada de cor compreendida de oxicarburetos metálicos refratários ricos em oxigênio, uma segunda camada de cor que compreende de oxicarburetos metálicos refratários ricos em oxigênio e uma camada de topo de óxidos metálicos refratários.' where numero='102014005512';
update titulo set resumo_cepit='Disclosed are non-woven gauze product and system for manufacturing the same. A non-woven cloth having rectangular holes in an array layout is formed by water jetting the fiber-web at the second spunlace machine. A non-woven cloth can be a substitute for the current gauze. Com a presente invenção, um tecido não tecido semelhante a gaze é alcançado utilizando o processo de fabricação de tecido não tecido de spunlace aqui divulgado, e 